In [1]:
import sys

In [2]:
sys.path

['C:\\Users\\User\\Desktop\\מדמח\\מודלימז\\פרויקט חדש',
 'C:\\Users\\User\\Downloads\\pytorch-normalizing-flows-master\\pytorch-normalizing-flows-master\\nflib',
 'C:\\Users\\User\\try',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\DLLs',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39',
 '',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\plotbrowser-0.1-py3.9.egg',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\win32',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\Pythonwin']

In [3]:
sys.path.append(r"c:\users\user\anaconda3\envs\pytorch_env\lib\site-packages")


In [4]:
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch

import torch.nn as nn
import torch.nn.functional as F
import time
from util import findStatistics
from sklearn.preprocessing import LabelEncoder

In [8]:
TRAIN_DATA=pd.read_csv(r"C:\Users\User\Desktop\מדמח\מודלימז\פרויקט חדש\CREDIT SCORE\train.csv")

TRAIN_DATA.drop(['ID', 'Customer_ID', 'Name', 'SSN', 'Type_of_Loan'], inplace=True, axis=1)
TRAIN_DATA.dropna(inplace=True)
def get_num_months(x: str) -> str:
    split_data = x.split(" ")
    return str(int(split_data[0]) * 12 + int(split_data[3]))


TRAIN_DATA['Credit_History_Age'] = TRAIN_DATA['Credit_History_Age'].map(get_num_months)
# Go through data and see what iregularities
TRAIN_DATA.replace('_', np.NaN, inplace=True)

TRAIN_DATA.replace('', np.NaN, inplace=True)

TRAIN_DATA.replace('_______', np.NaN, inplace=True)
TRAIN_DATA.dropna(inplace=True)
import re

# Go through ["Age", "Annual_Income", "Num_of_Loan", "Num_of_Delayed_Payment","Changed_Credit_Limit",
# "Num_Credit_Inquiries" "Outstanding_Debt","Amount_invested_monthly", "Monthly_Balance"] and strip the data and let
# only numeric

TRAIN_DATA['Age'] = TRAIN_DATA['Age'].map(lambda x: re.sub('\D', '', x))
TRAIN_DATA['Annual_Income'] = TRAIN_DATA['Annual_Income'].map(lambda x: re.sub('\D', '', x))
TRAIN_DATA['Num_of_Loan'] = TRAIN_DATA['Num_of_Loan'].map(lambda x: re.sub('\D', '', x))
TRAIN_DATA['Num_of_Delayed_Payment'] = TRAIN_DATA['Changed_Credit_Limit'].map(lambda x: re.sub('\D', '', x))
TRAIN_DATA['Changed_Credit_Limit'] = TRAIN_DATA['Changed_Credit_Limit'].map(lambda x: re.sub('\D', '', x))
TRAIN_DATA['Num_Credit_Inquiries'] = TRAIN_DATA['Outstanding_Debt'].map(lambda x: re.sub('\D', '', x))
TRAIN_DATA['Amount_invested_monthly'] = TRAIN_DATA['Amount_invested_monthly'].map(lambda x: re.sub('\D', '', x))
TRAIN_DATA['Monthly_Balance'] = TRAIN_DATA['Monthly_Balance'].map(lambda x: re.sub('\D', '', str(x)))
TRAIN_DATA['Outstanding_Debt'] = TRAIN_DATA['Outstanding_Debt'].map(lambda x: re.sub('\D', '', x))
# Convert numeric data to its appropriate type
TRAIN_DATA = TRAIN_DATA.astype({"Age": float, "Annual_Income": float, "Num_of_Loan": float, "Num_of_Delayed_Payment":
    float,
                                "Changed_Credit_Limit": float, "Num_Credit_Inquiries": float, "Outstanding_Debt": float,
                                "Amount_invested_monthly": float,
                                "Monthly_Balance": float})
TRAIN_DATA.drop(TRAIN_DATA[(TRAIN_DATA['Age'] > 80) | (TRAIN_DATA['Age'] < 18)].index, inplace=True)
# Drop some Monthly Inland outliers
TRAIN_DATA.drop(TRAIN_DATA[(TRAIN_DATA['Monthly_Inhand_Salary'] > 12500)].index,
                inplace=True)
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer

# Will just go with the default Ordinal Encoder

transformer_TRAIN = make_column_transformer(
    (OrdinalEncoder(), ['Month', 'Occupation', 'Credit_Mix', 'Payment_of_Min_Amount', 'Payment_Behaviour']),
    remainder='passthrough')
transformed_TRAIN_data = transformer_TRAIN.fit_transform(TRAIN_DATA)
transformed_TRAIN_data = pd.DataFrame(transformed_TRAIN_data, columns=transformer_TRAIN.feature_names_in_)
X = transformed_TRAIN_data.iloc[:, :22]
y = transformed_TRAIN_data.iloc[:, 22]
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.fit_transform(X_test)

In [9]:
parameters = {'max_depth':list(range(50)), 'min_samples_leaf':list(range(2,50,1)),'criterion':["gini", "entropy"]}

et=time.time()
model= DecisionTreeClassifier()
clf = GridSearchCV(model, parameters)

clf.fit(X_train, y_train)
ft=time.time()

print("elapsed time {}".format(ft-et))
print("score: {}".format(clf.score(X_test,y_test)))

elapsed time 8642.601002931595
score: 0.6628888631895455


In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(15,32)
        self.fc2=nn.Linear(32,64)
        self.fc6=nn.Linear(64,128)
        self.fc7=nn.Linear(128,256)
        self.fc8=nn.Linear(256,128)
        self.fc3=nn.Linear(128,32)
        self.fc4=nn.Linear(32,8)
        self.fc5=nn.Linear(8,1)


    def forward(self,x):
        
        x = F.relu(self.fc1(x))
        
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = F.relu(self.fc8(x))
        x = F.relu(self.fc3(x))
        
        x = F.relu(self.fc4(x))
        x = F.sigmoid(self.fc5(x))
        return x
    
     
    
    
net = Net()
net.load_state_dict(torch.load(r'C:\Users\User\Desktop\מדמח\מודלימז\פרויקט חדש\batches117'))
#best_max_depth,best_min_samples_leaf,best_criterion=params[np.argmax(scores)]
#best_criterion='entropy' if best_criterion==1 else 'gini'

<All keys matched successfully>

In [11]:
y_train=np.array(y_train)

In [12]:
scores=[]

et=time.time()
statistics=np.array(findStatistics(X_train,y_train))
for max_depth in list(range(50)):
    for min_samples_leaf in list(range(2,50,1)):
        for criterion in [1, 2]:
            hyperTensor=np.array([max_depth,min_samples_leaf,criterion])
            combtensor=torch.tensor(np.concatenate((statistics,hyperTensor))).float()
            
            score_predicted=net(combtensor).detach().numpy()
            scores.append(score_predicted)
params=[]
for max_depth in list(range(50)):
    for min_samples_leaf in list(range(2,50,1)):
        for criterion in [1, 2]:
            params.append((max_depth,min_samples_leaf,criterion)) 
            
scores=np.array(scores).flatten()
params=np.array(params)

tenLargetIdx = (-scores).argsort()[:10]
bests=params[tenLargetIdx]   


res=[]
for max_depth,min_samples_leaf,criterion in bests:
    criterion='entropy' if criterion==1 else 'gini'
    model= DecisionTreeClassifier(max_depth=max_depth,min_samples_leaf=min_samples_leaf,criterion=criterion)
    model.fit(X_train,y_train)
    res.append(model.score(X_test,y_test))
    

best=np.argmax(np.array(res))
best_max_depth,best_min_samples_leaf,best_criterion=bests[best]
best_criterion='entropy' if best_criterion==1 else 'gini'
model2= DecisionTreeClassifier(max_depth=best_max_depth,min_samples_leaf=best_min_samples_leaf,criterion=best_criterion)

model2.fit(X_train,y_train)
ft=time.time()
print("elapsed time {}".format(ft-et))

print('score: {}'.format(model2.score(X_test,y_test)))
            


elapsed time 39.79829716682434
score: 0.6678616861339193


In [17]:
best_max_depth

8